In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [6]:
station = db.get_station_by_code(session, '91')

In [12]:
[m for m in station.monthly_measurements if m.date.month in (1, 2, 3) and m.variable.name == "TDd"]

[MonthlyMeasurement(date=datetime.date(1987, 1, 1), id=UUID('0f78a845-0db1-45ef-bac8-03f8d68ff034'), variable_id=UUID('32706ce0-0134-4122-91a0-9d6f237350b3'), value=-8.23, station_id=UUID('8d9e6fe2-b79d-448f-bf33-b4a9d9b17e35')),
 MonthlyMeasurement(date=datetime.date(1988, 1, 1), id=UUID('cbe5ba03-9996-44b8-b707-1b29b1fb309f'), variable_id=UUID('32706ce0-0134-4122-91a0-9d6f237350b3'), value=-4.34, station_id=UUID('8d9e6fe2-b79d-448f-bf33-b4a9d9b17e35')),
 MonthlyMeasurement(date=datetime.date(1989, 1, 1), id=UUID('103497ee-ec72-4b1b-8992-2e1aea3270a3'), variable_id=UUID('32706ce0-0134-4122-91a0-9d6f237350b3'), value=-2.42, station_id=UUID('8d9e6fe2-b79d-448f-bf33-b4a9d9b17e35')),
 MonthlyMeasurement(date=datetime.date(1990, 1, 1), id=UUID('761f4b69-18f7-4364-9660-88a5813d15d7'), variable_id=UUID('32706ce0-0134-4122-91a0-9d6f237350b3'), value=-3.319, station_id=UUID('8d9e6fe2-b79d-448f-bf33-b4a9d9b17e35')),
 MonthlyMeasurement(date=datetime.date(1991, 1, 1), id=UUID('bff40fed-5a6c-493b